In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from numpy import random
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import time
import re
from dateutil.parser import parse
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

sns.set_palette('Dark2')
plt.rcParams['figure.figsize'] = (8,6)
%matplotlib inline


In [2]:
numFiles=321
dataStarts = (np.linspace(5500,5500+20*(numFiles-1),numFiles))
frames = []
for n in range(len(dataStarts)):
    frames.append(pd.read_csv('../WineData/wineData_'+str(int(dataStarts[n]))+'_'+str(int(dataStarts[n]+19))+'.csv',index_col='Name'))
wines1 = pd.concat(frames,ignore_index=False).drop('Unnamed: 0',axis=1)
# print len(wines1)
wines1.head()

,WineMaker,Variety,Origin,Alcohol (%vol),pH,ResidualSugar (g/l),TA (g/l),TastingNotes,Ageing,BlendPercent
Name,,,,,,,,,,
Van Loveren Pinot Gris 2002,NaN,Pinot Gris,Robertson,13.5 %vol,3.5,1.8 g/l,6.4 g/l,Pinot Gris is a full bodied well balanced wine...,1-3 years,Pinot Gris
Van Loveren Limited Release Chardonnay 2002,NaN,Chardonnay,Robertson,13.5 %vol,3.4,1.7 g/l,6.3 g/l,"A full-bodied, well-balanced, nutty wine that ...",1-5 years,Chardonnay
Whalehaven Baleine Noir NV,NaN,NaN,Walker Bay,NaN,NaN,NaN,NaN,This special release is a lightly wooded blend...,NaN,Blend
Indalo Shiraz 2001,Andries Blake,Shiraz,Coastal,15.0 %vol,3.5,2.5 g/l,5.2 g/l,"Intense dark ruby red colour. Spicy, saddle ba...",This wine will improve in complexity over the ...,100% Shiraz
Indalo Pinotage,Andries Blake,Pinotage,Coastal,13.25 %vol,3.73,1.7 g/l,5.39 g/l,Michelangelo Awards 2005 - Silver MedalVery da...,NaN,Pinotage


In [3]:
wines1.describe(include = 'all')

,WineMaker,Variety,Origin,Alcohol (%vol),pH,ResidualSugar (g/l),TA (g/l),TastingNotes,Ageing,BlendPercent
count,4015,4633,3967,4133,3923.0,3992,3923,3861,1616,4614
unique,430,44,49,521,200.0,703,361,3660,1101,998
top,Frans Smit,Cabernet Sauvignon,Coastal,13.5 %vol,3.5,2.5 g/l,5.8 g/l,Available in Holland only,A multi-dimensional wine exhibiting excellent ...,100% Chardonnay
freq,88,789,1866,480,196.0,154,235,6,20,466


In [4]:
wines2 = pd.read_csv('../WineData/winecoza.csv',sep=';',index_col='Wine')
# print len(wines2)
wines2.head()

,CommonName,Latitude,Longitude,Vintage,Origin,TastingNotes,Alcohol,RS,pH,TA,VA,TSO2,FSO2,MainVariety,WineType,WineStyle,WineBody,WineTaste,Wooded
Wine,,,,,,,,,,,,,,,,,,,
The Bernard Series Hand Picked Viognier 2015,Bellingham Wines,"-33,52944183","19,01231003",2015,Coastal Region,Luminescent gold brushed by youthful green gli...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Viognier,White,Dry,Full,NaN,NaN
Durbanville Hills Vineyard Luipaardsberg Selection Merlot 2011,Durbanville Hills Cellar,"-33,82494354","18,56601906",2011,Durbanville,Colour: Ruby red.<br />Nose: Concentration of ...,14.5,2.9,3.64,5.6,NaN,NaN,NaN,Merlot,Red,Dry,Full,NaN,NaN
GÃ´iya Chardonnay / Sauvignon Blanc 2011,Namaqua Wines,0,0,2011,NaN,"For more information about the brand, please v...",13.0,7.38,3.5,6.19,0.36,NaN,NaN,Chardonnay,White,Off Dry,NaN,NaN,NaN
GÃ´iya Shiraz / Pinotage 2011,Namaqua Wines,0,0,2011,NaN,"For more information about the brand, please v...",14.0,5.93,3.53,5.8,0.49,NaN,NaN,Shiraz,Red,NaN,NaN,NaN,NaN
The FMC 2014,Ken Forrester Wines,"-34,02584839","18,81930923",2014,Stellenbosch,The pinnacle of what we consider to be the fin...,14.0,7.1,3.37,6.8,NaN,NaN,NaN,Chenin Blanc,White,NaN,NaN,NaN,NaN


In [5]:
wines2.describe(include='all')

,CommonName,Latitude,Longitude,Vintage,Origin,TastingNotes,Alcohol,RS,pH,TA,VA,TSO2,FSO2,MainVariety,WineType,WineStyle,WineBody,WineTaste,Wooded
count,3097,3057,3057,3097.000000,2919,3075,2990,2944,2861,2838,407,457.000000,407.000000,3088,3097,2503,1725,1707,0.0
unique,342,251,253,NaN,88,3008,486,596,147,279,83,NaN,NaN,47,10,7,6,5,NaN
top,Fairview Wine & Cheese,0,0,NaN,Stellenbosch,Flavour profiles vary between ripe apple and p...,14.0,2.7,3.5,5.8,0.45,NaN,NaN,Shiraz,Red,Dry,Full,Fruity,NaN
freq,50,279,279,NaN,730,4,380,107,155,211,21,NaN,NaN,452,1510,2125,846,1252,NaN
mean,NaN,NaN,NaN,2013.465289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.811138,32.721990,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,1.454357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.416257,13.453557,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,2011.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,2012.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.000000,25.500000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,2014.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.000000,33.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,2015.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.000000,39.000000,NaN,NaN,NaN,NaN,NaN,NaN


# Clean data

Although the tasting notes are one of the major focuses of the project, I look at leaning all of the other data first...

## Clean wines1

From exploration, the data scraped from the website has the following issues:
- Alcohol (%vol), ResidualSugar (g/l) and	TA (g/l) contain units 
- Headings are multiple words

In [6]:
wines1 = wines1.rename(columns = {'Alcohol (%vol)':'Alcohol','ResidualSugar (g/l)':'RS','TA (g/l)':'TA'})
wines1.Alcohol = wines1.Alcohol.str.rstrip('\xc2\xa0%vol').str.replace(',','.').astype(float)
wines1.RS = wines1.RS.str.replace('Dry','99')
wines1.RS = wines1.RS.str.rstrip(r' g/l').str.replace(',','.')
wines1.RS = wines1.RS.apply(pd.to_numeric, errors='coerce')
wines1.RS = wines1.RS.replace(99,np.nan)
wines1.TA = wines1.TA.str.rstrip(r' g/l').str.replace(',','.')
wines1.TA = wines1.TA.apply(pd.to_numeric, errors='coerce')
wines1.pH = wines1.pH.str.replace(',','.').astype(float)
wines1.pH = wines1.pH.apply(pd.to_numeric, errors='coerce')

Use the wine name as the index

In [7]:
wines1['WineName'] = wines1.index
wines1 = wines1.reset_index(drop=True)

Get the year of the wine from the name

In [8]:
def returnYear(s):
    s = s.WineName
    try:
        yr = parse(s, fuzzy=True).year   #re.search(r" \d{4}", s).group(1)
    except:
        yr = np.nan
    return yr 

wines1['Year'] = wines1.apply(returnYear,axis= 1)
wines1.head()

,WineMaker,Variety,Origin,Alcohol,pH,RS,TA,TastingNotes,Ageing,BlendPercent,WineName,Year
0,NaN,Pinot Gris,Robertson,13.50,NaN,1.8,6.40,Pinot Gris is a full bodied well balanced wine...,1-3 years,Pinot Gris,Van Loveren Pinot Gris 2002,2002.0
1,NaN,Chardonnay,Robertson,13.50,NaN,1.7,6.30,"A full-bodied, well-balanced, nutty wine that ...",1-5 years,Chardonnay,Van Loveren Limited Release Chardonnay 2002,2002.0
2,NaN,NaN,Walker Bay,NaN,NaN,NaN,NaN,This special release is a lightly wooded blend...,NaN,Blend,Whalehaven Baleine Noir NV,NaN
3,Andries Blake,Shiraz,Coastal,15.00,NaN,2.5,5.20,"Intense dark ruby red colour. Spicy, saddle ba...",This wine will improve in complexity over the ...,100% Shiraz,Indalo Shiraz 2001,2001.0
4,Andries Blake,Pinotage,Coastal,13.25,NaN,1.7,5.39,Michelangelo Awards 2005 - Silver MedalVery da...,NaN,Pinotage,Indalo Pinotage,NaN


## Clean wines2

From exploration, the data given by wines.co.za has the following issues:

- Alcohol, RS, TA and pH all contain strange characters and ranges. I could force all to floats but would lose much of my data
- VA is volatile acidity

In [9]:
def cleanRange(dat):
    if isinstance(dat,str):
        if '-' in dat:
            d = dat.split('-')
            return (float(d[0])+float(d[1]))/2
def cleaningData(series):
    series = series.str.replace('*','')
    series = series.str.replace('`','')
    series = series.str.replace('/','')
#     series = series.str.replace(' ','')
    series = series.str.replace('>','')
    series = series.str.replace('<','')
    series = series.str.replace(',','.')
    series = series.apply(lambda x: cleanRange(x))
    return series.astype(float)

In [10]:
wines2.Alcohol = cleaningData(wines2.Alcohol)
wines2.WineType.str.replace('Cap Classi','Cap Classique')
wines2.RS = cleaningData(wines2.RS)
wines2.TA = cleaningData(wines2.TA)
wines2.pH = cleaningData(wines2.pH)

Use the wine name as the index

In [11]:
wines2['WineName'] = wines2.index
wines2 = wines2.reset_index(drop=True)

## Merge wine data
Now, I need to fill in missing data between databases where possible. For example, the name or variety can tell me if the wine is red or white. 

There are also a few repeat entries to merge

In [110]:
wines= pd.merge(wines1, wines2, on='WineName', how='outer')
wines.head()

,WineMaker,Variety,Origin_x,Alcohol_x,pH_x,RS_x,TA_x,TastingNotes_x,Ageing,BlendPercent,...,TA_y,VA,TSO2,FSO2,MainVariety,WineType,WineStyle,WineBody,WineTaste,Wooded
0,NaN,Pinot Gris,Robertson,13.50,NaN,1.8,6.40,Pinot Gris is a full bodied well balanced wine...,1-3 years,Pinot Gris,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Chardonnay,Robertson,13.50,NaN,1.7,6.30,"A full-bodied, well-balanced, nutty wine that ...",1-5 years,Chardonnay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Walker Bay,NaN,NaN,NaN,NaN,This special release is a lightly wooded blend...,NaN,Blend,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andries Blake,Shiraz,Coastal,15.00,NaN,2.5,5.20,"Intense dark ruby red colour. Spicy, saddle ba...",This wine will improve in complexity over the ...,100% Shiraz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andries Blake,Pinotage,Coastal,13.25,NaN,1.7,5.39,Michelangelo Awards 2005 - Silver MedalVery da...,NaN,Pinotage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
wines['Origin_x'].fillna(wines['Origin_y'], inplace=True)
wines['Alcohol_x'].fillna(wines['Alcohol_y'], inplace=True)
wines['pH_x'].fillna(wines['pH_y'], inplace=True)
wines['RS_x'].fillna(wines['RS_y'], inplace=True)
wines['TA_x'].fillna(wines['TA_y'], inplace=True)
wines['TastingNotes_x'].fillna(wines['TastingNotes_y'], inplace=True)
wines['MainVariety'].fillna(wines['Variety'], inplace=True)
wines['Vintage'].fillna(wines['Year'], inplace=True)
del wines['Origin_y'];del wines['Year'];del wines['Alcohol_y'];del wines['pH_y'];del wines['RS_y']
del wines['TA_y'];del wines['TastingNotes_y'];del wines['Variety']
wines = wines.rename(columns = {'Alcohol_x':'Alcohol','RS_x':'RS','TA_x':'TA','pH_x':'pH','TastingNotes_x':'TastingNotes','Origin_x':'Origin'})

In [114]:
def assignWineType(row):
    '''
    Assign the right type to each wine given the Varietal or Name. The name ovverides the varietal so that a dessert red will be labelled as dessert rather than red
    '''
    TypeVarietal = {'Red':['Pinotage', 'Cabernet Sauvignon','Shiraz',
        'Merlot', 'Mourvedre','Cinsaut','Ruby Cabernet', 'Tinta Barocca', 'Touriga Nacional', 'Souzao',
       'Cabernet Franc', 'Barbera','Pinot Noir', 'Blend - Red',
        'Malbec', 'Grenache', 'Nebbiolo', 'Zinfandel', 
       'Sangiovese','Petit Verdot', 'Tempranillo',
       'Carignan',  'Tannat','Gamay Noir', 'Muscat de Hambourg'],
       'White':['Colombard','Furmint', 'Blend - White', 'Muscat Ottonel', 'Grenache blanc','Palomino','Harslevel\xc3\x83\xc2\xbc', 'Sylvaner','Sultana','Morio Muscat','Clairette Blanche', 'Durif','Cape Riesling', 'Verdelho','Weisser Riesling', 'Muscat dAlexandrie', 'Roussanne','Chenin Blanc','Bukettraube','Riesling', 'Muscat de Frontignan', 'Pinot Gris','Gewurztraminer', 'Sauvignon Blanc','Viognier','Semillon','Chardonnay'],
        'Rose':['Blend - Rose'],
        'Fortified':['Fortified'], 
        'Sparkling':['Brut','Sparkling'],
       'Dessert':['Muscadel','Dessert'], 
        'Cap Classique':['Cap Classique'],
        'Spirit':['Spirit','0','Sherry','Brandy','Boegoeblits'],
        'Other':['Olive oil']}
    if isinstance(row.WineType,float):
        for k in range(len(TypeVarietal.keys())):
            for v in TypeVarietal.values()[k]:
                if isinstance(row.MainVariety,float):
                    pass
                elif v in row.MainVariety: 
                    return TypeVarietal.keys()[k]
        for k in range(len(TypeVarietal.keys())):
            for v in TypeVarietal.values()[k]:
                if v in row.WineName: 
                    return TypeVarietal.keys()[k]
    elif row.WineType == '0':
        row.WineType = 'Spirit'
    else:
        return row.WineType
    if isinstance(row.MainVariety,float):
        for k in range(len(TypeVarietal.keys())):
            for v in TypeVarietal.values()[k]:
                if v in row.WineName: 
                    return TypeVarietal.keys()[k]

In [115]:
wines['Vintage_dateTime'] = wines.Vintage.fillna(0).astype(int)
wines.Vintage_dateTime = pd.to_datetime(wines.Vintage_dateTime,format='%Y', errors = 'coerce')
wines.WineType = wines.apply(lambda x :assignWineType(x),axis=1)
wines.WineType.unique()

array(['White', None, 'Red', 'Spirit', 'Sparkling', 'Rose', 'Dessert',
       'Fortified', 'Cap Classi', 'Cap Classique'], dtype=object)

In [116]:
wines.describe(include = 'all')

,WineMaker,Origin,Alcohol,pH,RS,TA,TastingNotes,Ageing,BlendPercent,WineName,...,VA,TSO2,FSO2,MainVariety,WineType,WineStyle,WineBody,WineTaste,Wooded,Vintage_dateTime
count,4015,6886,4144.000000,77.000000,3950.00000,3918.000000,6936,1616,4614,7833,...,407,457.000000,407.000000,7721,7775,2503,1725,1707,0.0,7468
unique,430,109,NaN,NaN,NaN,NaN,6661,1101,998,7782,...,83,NaN,NaN,56,9,7,6,5,NaN,28
top,Frans Smit,Coastal,NaN,NaN,NaN,NaN,Available in Holland only,A multi-dimensional wine exhibiting excellent ...,100% Chardonnay,Rust en Vrede Estate Merlot 2003,...,0.45,NaN,NaN,Cabernet Sauvignon,Red,Dry,Full,Fruity,NaN,2003-01-01 00:00:00
freq,88,1879,NaN,NaN,NaN,NaN,6,20,466,3,...,21,NaN,NaN,1198,4069,2125,846,1252,NaN,1314
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01 00:00:00
mean,NaN,NaN,13.718962,3.475455,9.54576,6.075804,NaN,NaN,NaN,NaN,...,NaN,101.811138,32.721990,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,1.902736,0.197454,29.99486,6.368910,NaN,NaN,NaN,NaN,...,NaN,33.416257,13.453557,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,3.030000,0.00000,0.240000,NaN,NaN,NaN,NaN,...,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,13.000000,3.390000,2.03250,5.500000,NaN,NaN,NaN,NaN,...,NaN,84.000000,25.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now I want to remove duplicate wines

In [117]:
def common_start(*strings):    #https://stackoverflow.com/questions/18715688/find-common-substring-between-two-strings
    """ Returns the longest common substring
        from the beginning of the `strings`
    """
    def _iter():
        for z in zip(*strings):
            if z.count(z[0]) == len(z):  # check all elements in `z` are the same
                yield z[0]
            else:
                return

    return ''.join(_iter())

def merge(x):
    if len(set(x.apply(type)))==1:
        if isinstance(x.iloc[0],float):
            return x.mean()
        if isinstance(x.iloc[0],str):
            if len(set(x)) ==1:
                return x.iloc[0]
            else:
                lens = x.str.len()
                if len(set(lens))==1:
                    return x.iloc[0]
                else:
                    pass
                    return x.apply(common_start)
    else:
        return merge(x.dropna())
#     print type(x.iloc[0]),x.iloc[0]
        

def mergeRows(rows):
    if len(rows)>1:
        mergedRow = rows.apply(lambda x: merge(x))
        return mergedRow
    else:
        return rows

wines = wines.groupby('WineName',sort=False).apply(lambda x: mergeRows(x))
del wines[0]  # this function adds a new column with the label 0, and we dont want to keep it
wines.reset_index(inplace=True,drop=True)

In [118]:
wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8777 entries, 0 to 8776
Data columns (total 24 columns):
Ageing              1592 non-null object
Alcohol             4080 non-null float64
BlendPercent        4541 non-null object
CommonName          3085 non-null object
FSO2                402 non-null float64
Latitude            3046 non-null object
Longitude           3046 non-null object
MainVariety         7634 non-null object
Origin              6809 non-null object
RS                  3892 non-null float64
TA                  3858 non-null float64
TSO2                452 non-null float64
TastingNotes        6864 non-null object
VA                  404 non-null object
Vintage             7392 non-null float64
Vintage_dateTime    7382 non-null object
WineBody            1716 non-null object
WineMaker           3949 non-null object
WineName            7745 non-null object
WineStyle           2493 non-null object
WineTaste           1697 non-null object
WineType            7687 non-

In [119]:
#Remove all empty rows where the wine name is missing
wines = wines.drop(wines.loc[wines.WineName.isnull()].index)

In [120]:
wines.to_csv('../WineData/cleanWineData.csv', columns = wines.columns, encoding='utf-8')

In [121]:
wines.head()

,Ageing,Alcohol,BlendPercent,CommonName,FSO2,Latitude,Longitude,MainVariety,Origin,RS,...,Vintage,Vintage_dateTime,WineBody,WineMaker,WineName,WineStyle,WineTaste,WineType,Wooded,pH
0,1-3 years,13.50,Pinot Gris,NaN,NaN,NaN,NaN,Pinot Gris,Robertson,1.8,...,2002.0,2002-01-01 00:00:00,NaN,NaN,Van Loveren Pinot Gris 2002,NaN,NaN,White,NaN,NaN
1,1-5 years,13.50,Chardonnay,NaN,NaN,NaN,NaN,Chardonnay,Robertson,1.7,...,2002.0,2002-01-01 00:00:00,NaN,NaN,Van Loveren Limited Release Chardonnay 2002,NaN,NaN,White,NaN,NaN
2,NaN,NaN,Blend,NaN,NaN,NaN,NaN,NaN,Walker Bay,NaN,...,NaN,NaN,NaN,NaN,Whalehaven Baleine Noir NV,NaN,NaN,None,NaN,NaN
3,This wine will improve in complexity over the ...,15.00,100% Shiraz,NaN,NaN,NaN,NaN,Shiraz,Coastal,2.5,...,2001.0,2001-01-01 00:00:00,NaN,Andries Blake,Indalo Shiraz 2001,NaN,NaN,Red,NaN,NaN
4,NaN,13.25,Pinotage,NaN,NaN,NaN,NaN,Pinotage,Coastal,1.7,...,NaN,NaN,NaN,Andries Blake,Indalo Pinotage,NaN,NaN,Red,NaN,NaN
